## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [17]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,36.18,72,100,2.35,overcast clouds
1,1,Oranjestad,AW,12.5240,-70.0270,85.95,66,40,17.27,scattered clouds
2,2,Rikitea,PF,-23.1203,-134.9692,76.44,75,0,21.07,clear sky
3,3,Dikson,RU,73.5069,80.5464,-13.23,100,100,8.41,overcast clouds
4,4,Andenes,NO,69.3143,16.1194,19.38,68,20,6.91,few clouds


In [18]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [19]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Oranjestad,AW,12.5240,-70.0270,85.95,66,40,17.27,scattered clouds
2,2,Rikitea,PF,-23.1203,-134.9692,76.44,75,0,21.07,clear sky
8,8,Kuching,MY,1.5500,110.3333,77.22,94,20,2.30,few clouds
11,11,Puerto Ayora,EC,-0.7393,-90.3518,78.76,83,91,9.73,overcast clouds
12,12,Cockburn Town,TC,21.4612,-71.1419,79.30,70,100,12.19,overcast clouds
16,16,Puerto Leguizamo,CO,-0.1934,-74.7819,76.10,94,83,3.60,broken clouds
19,19,Conceicao Do Araguaia,BR,-8.2578,-49.2647,78.66,86,42,4.63,scattered clouds
22,22,Castro,BR,-24.7911,-50.0119,75.63,48,0,3.02,clear sky
26,26,Mahebourg,MU,-20.4081,57.7000,77.29,78,40,2.30,scattered clouds
29,29,Atuona,PF,-9.8000,-139.0333,78.60,76,5,19.39,clear sky


In [20]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                218
City                   218
Country                217
Lat                    218
Lng                    218
Max Temp               218
Humidity               218
Cloudiness             218
Wind Speed             218
Current Description    218
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                217
City                   217
Country                217
Lat                    217
Lng                    217
Max Temp               217
Humidity               217
Cloudiness             217
Wind Speed             217
Current Description    217
dtype: int64

In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Oranjestad,AW,85.95,scattered clouds,12.5240,-70.0270,
2,Rikitea,PF,76.44,clear sky,-23.1203,-134.9692,
8,Kuching,MY,77.22,few clouds,1.5500,110.3333,
11,Puerto Ayora,EC,78.76,overcast clouds,-0.7393,-90.3518,
12,Cockburn Town,TC,79.30,overcast clouds,21.4612,-71.1419,
16,Puerto Leguizamo,CO,76.10,broken clouds,-0.1934,-74.7819,
19,Conceicao Do Araguaia,BR,78.66,scattered clouds,-8.2578,-49.2647,
22,Castro,BR,75.63,clear sky,-24.7911,-50.0119,
26,Mahebourg,MU,77.29,scattered clouds,-20.4081,57.7000,
29,Atuona,PF,78.60,clear sky,-9.8000,-139.0333,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(hotels["results"][0]["name"] + " found, near " + row["City"])
    except:
        print("Hotel not found... skipping.")
        

Divi Village Golf & Beach Resort found, near Oranjestad
People ThankYou found, near Rikitea
The Ranee Boutique Suites found, near Kuching
Finch Bay Galapagos Hotel found, near Puerto Ayora
Osprey Beach Hotel found, near Cockburn Town
la casona de juancho found, near Puerto Leguizamo
Tarumã Tropical Hotel found, near Conceicao Do Araguaia
CHACARA BAILLY found, near Castro
Shandrani Beachcomber Resort & Spa found, near Mahebourg
Villa Enata found, near Atuona
Hotel Sri Krishna Residency found, near Malpe
INN NEW HORIZON found, near Touros
Maui Seaside Hotel found, near Kahului
Hotel Olako found, near Omboue
Kost Pria found, near Meulaboh
Rainforest Haven Inn found, near San Ignacio
Kin Ha Tulum found, near Tulum
Zango lodge found, near Senanga
Tanna Lodge found, near Isangel
UN Humanitarian Hub found, near Bama
Bomberos voluntarios castaños coahuila found, near Castanos
Hotels in Lakshadweep Islands found, near Kavaratti
Oustaz Vame DIAKITE found, near Odienne
Pousada Porto Praia found, 

Holiday Inn Express & Suites Port Lavaca found, near Port Lavaca
Hotel not found... skipping.
Villa Orphee found, near Les Cayes
Banjanay Hotel found, near Xuddur
Hawthorn Suites By Wyndham Las Vegas/Henderson found, near Henderson
Hotel not found... skipping.
Vahmoneh Lodge Vanimo found, near Vanimo
Hotel Pigalle found, near Cabedelo
CeBlue Villas & Beach Resort found, near The Valley
Hotel Aconcagua found, near Presidencia Roque Saenz Pena
Solmar Gracious Hotel found, near Lodwar
Motel 6 Sinton, TX found, near San Patricio
Pousada do Ninho found, near America Dourada
Pousada Monte Carmelo O Pão found, near Belmonte
Hotel not found... skipping.
Cattleman's Rest Motor Inn found, near Charters Towers
Royal Suite found, near Bida
Complejo Turístico Farelluk found, near San Vicente
Sing and Swing Lodge found, near Balabac
JABIR HOTEL found, near Bandarbeyla
Sama Emirate Hotel found, near Aden
Hospedaje Paraíso found, near Tingo Maria


In [24]:
hotel_df.count()

City                   217
Country                217
Max Temp               217
Current Description    217
Lat                    217
Lng                    217
Hotel Name             217
dtype: int64

In [25]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Oranjestad,AW,85.95,scattered clouds,12.5240,-70.0270,Divi Village Golf & Beach Resort
2,Rikitea,PF,76.44,clear sky,-23.1203,-134.9692,People ThankYou
8,Kuching,MY,77.22,few clouds,1.5500,110.3333,The Ranee Boutique Suites
11,Puerto Ayora,EC,78.76,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
12,Cockburn Town,TC,79.30,overcast clouds,21.4612,-71.1419,Osprey Beach Hotel
16,Puerto Leguizamo,CO,76.10,broken clouds,-0.1934,-74.7819,la casona de juancho
19,Conceicao Do Araguaia,BR,78.66,scattered clouds,-8.2578,-49.2647,Tarumã Tropical Hotel
22,Castro,BR,75.63,clear sky,-24.7911,-50.0119,CHACARA BAILLY
26,Mahebourg,MU,77.29,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
29,Atuona,PF,78.60,clear sky,-9.8000,-139.0333,Villa Enata


In [26]:
clean_hotel_df.count()

City                   205
Country                205
Max Temp               205
Current Description    205
Lat                    205
Lng                    205
Hotel Name             205
dtype: int64

In [27]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))